In [1]:
from transformers import BertModel,BertTokenizer
import torch

In [2]:
pretrained_weights = 'bert-base-uncased'

In [3]:
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

## Loading dataframe files

caption.json            msrvtt_new_info.json
info.json               videodatainfo_2017.json


In [4]:
import json

In [5]:
captions = json.load(open('./data/caption.json'))
info = json.load(open('./data/info.json'))
ix_to_word = info['ix_to_word']
word_to_ix = info['word_to_ix']
splits = info['videos']

In [6]:
sample = captions['video0']['captions'][10]
    

In [7]:
sample

'a man showing the various features of a car'

In [8]:
encoded_sample = torch.tensor(tokenizer.encode(sample)).unsqueeze(0)

In [9]:
model = BertModel.from_pretrained(pretrained_weights)

In [10]:
encoded_sample

tensor([[ 101, 1037, 2158, 4760, 1996, 2536, 2838, 1997, 1037, 2482,  102]])

In [21]:
output = model(encoded_sample)

In [26]:
output[1].shape

torch.Size([1, 768])

In [20]:
output[:,0,:].shape ## Sentence level feature

torch.Size([1, 768])

In [34]:

from tqdm import tqdm
sent_feats = np.zeros((len(captions),768))

for i in tqdm(range(len(captions))):
    
    with torch.no_grad():
        ix = random.randint(0,19)
        cap = captions[f'video{i}']['captions'][ix]

        encoded_cap = torch.tensor(tokenizer.encode(cap)).unsqueeze(0)

        cap_feats = model(encoded_cap)[0] ## Word Prediction scores before softmax
        sent_feats[i,:] = cap_feats[:,0,:] ## Taking the <CLS> token embedding
    
    





100%|██████████| 10000/10000 [09:00<00:00, 18.50it/s]


In [48]:
!touch bert_caps.pkl

In [49]:
import pickle
with open("bert_caps.pkl","wb") as f:
    pickle.dump(sent_feats,f)

In [50]:
a = pickle.load(open("bert_caps.pkl","rb"))

## Building the Dataloader

In [12]:
from torch.utils.data import Dataset
import numpy as np
import torch
import random
import os

In [ ]:
class MSR_Dataset(Dataset):
    
    def __init__(self):
        self.captions = json.load(open('./data/caption.json'))
        self.info = json.load(open('./data/info.json'))
        self.ix_to_word = info['ix_to_word']
        self.word_to_ix = info['word_to_ix']
        self.splits = info['videos']
        
        